# Flowline Data

## Importing data and libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mapbox import Uploader

%matplotlib inline

In [3]:
%cd Flowline

/Users/Jackie/Desktop/Flowline


In [4]:
flowline = pd.read_csv('FlowlineInventory.csv', encoding = "ISO-8859-1", 
                       dtype = {'Integrity Test Results': str, 
                       'Well API County Code': str, 'Well Operator': str}, 
                       na_values = 'NaN', index_col = 0, low_memory = False)

## Check out the data

In [336]:
flowline.head()

,Import Date,Submitting Operator,Well Operator,Well API County Code,Well API Sequence Code,Flowline Riser Type,Start Riser Latitude,Start Riser Longitude,Start Municipality,Start Point Active,...,End Point Municipality,End Point Active,End Scheduled For Abandonment,End Point Comment,Integrity Test Method,Integrity Test Date,Integrity Test Results,Integrity Test Action,Integrity Test Comment,Line Abandoned
ID,,,,,,,,,,,,,,,,,,,,,
1,5/19/17,AEON ENERGY CORP,AEON ENERGY CORP,115,06058,compressor,40.95584,-102.54253,NaN,YES,...,NaN,NaN,NaN,NaN,pressure test,NaN,Pass,NaN,NaN,NaN
2,5/19/17,AEON ENERGY CORP,AEON ENERGY CORP,115,06058,meter,40.95584,-102.54253,NaN,YES,...,NaN,NaN,NaN,NaN,pressure test,NaN,Pass,NaN,NaN,NaN
3,5/19/17,AEON ENERGY CORP,AEON ENERGY CORP,115,06058,separator,40.95584,-102.54253,NaN,YES,...,NaN,NaN,NaN,NaN,pressure test,NaN,Pass,NaN,NaN,NaN
4,5/19/17,AEON ENERGY CORP,AEON ENERGY CORP,115,06058,wellhead riser,40.95584,-102.54253,NaN,YES,...,NaN,NaN,NaN,NaN,pressure test,NaN,Pass,NaN,NaN,NaN
5,5/19/17,AEON ENERGY CORP,AEON ENERGY CORP,115,05045,compressor,40.95585,-102.53921,NaN,YES,...,NaN,NaN,NaN,NaN,pressure test,NaN,Pass,NaN,NaN,NaN


In [337]:
flowline.columns

Index(['Import Date', 'Submitting Operator', 'Well Operator',
       'Well API County Code', 'Well API Sequence Code', 'Flowline Riser Type',
       'Start Riser Latitude', 'Start Riser Longitude', 'Start Municipality',
       'Start Point Active', 'Start Scheduled For Abandonment',
       'Start Comment', 'End Point Latitude', 'End PointLongitude',
       'End Point Municipality', 'End Point Active',
       'End Scheduled For Abandonment', 'End Point Comment',
       'Integrity Test Method', 'Integrity Test Date',
       'Integrity Test Results', 'Integrity Test Action',
       'Integrity Test Comment', 'Line Abandoned'],
      dtype='object')

In [338]:
print(flowline.describe())

       Start Riser Latitude  Start Riser Longitude  End Point Latitude  \
count          2.735100e+05          273504.000000        3.089620e+05   
mean           1.192936e+03            -105.110075        1.060681e+03   
std            1.285569e+05               1.746219        1.209565e+05   
min           -1.081236e+02            -140.603000       -1.081237e+02   
25%            4.006736e+01            -104.939560        4.007796e+01   
50%            4.017946e+01            -104.815740        4.017701e+01   
75%            4.030839e+01            -104.686290        4.028684e+01   
max            1.433487e+07             -10.454476        1.433485e+07   

       End PointLongitude  
count       308965.000000  
mean          -105.037913  
std              3.035536  
min          -1104.837335  
25%           -104.930752  
50%           -104.815650  
75%           -104.693830  
max              0.000000  


In [339]:
flowline_dup = flowline.duplicated()
print(flowline_dup.value_counts())

False    210427
True     106800
dtype: int64


### Pass, Fail, and No test were reported many different ways. This code just combines the different ways of stating 'Pass', 'Fail', and 'No test'

In [345]:
print(flowline['Integrity Test Results'].value_counts())

Pass              108308
PASS               31489
Fail                 490
FAIL                 302
pass                 189
Yes                  158
No                    14
Not Applicable        12
No test                3
fail                   2
-                      2
P                      1
Name: Integrity Test Results, dtype: int64


In [346]:
# Replace NaN with 'No test'
flowline['Integrity Test Results'] = flowline['Integrity Test Results'].fillna('No Data')
flowline['Integrity Test Results'].astype(str)
print(flowline['Integrity Test Results'].value_counts())

No Data           176257
Pass              108308
PASS               31489
Fail                 490
FAIL                 302
pass                 189
Yes                  158
No                    14
Not Applicable        12
No test                3
fail                   2
-                      2
P                      1
Name: Integrity Test Results, dtype: int64


# All lines (Assuming unique IDs are unique wells)

In [344]:
print('Total Wells Assuming All Are Unique:', flowline.shape[0])

Total Wells Assuming All Are Unique: 317227


In [368]:
flowline['Integrity Test Results'] = flowline['Integrity Test Results'].replace(
                                {'PASS': 'Pass', 'pass':'Pass', 'Yes':'Pass', 'P':'Pass', 
                                 'FAIL':'Fail', 'No':'Fail', 'fail':'Fail',
                                 'Not Applicable':'No Data', '-':'No Data', 
                                 'NaN':'No Data', 'No test':'No Data'})
print('Test Results For All Wells Assuming All Are Unique:')
print(flowline['Integrity Test Results'].value_counts())

Test Results For All Wells Assuming All Are Unique:
No Data    176274
Pass       140145
Fail          808
Name: Integrity Test Results, dtype: int64


## Broomfield (County Code 14)

In [349]:
print('Number of Wells in Broomfield:', 
      flowline['Integrity Test Results'][flowline['Well API County Code'] == 
                                         '014'].count())

Number of Wells in Broomfield: 643


In [363]:
print('Test Results for Broomfield Wells:')
print(flowline['Integrity Test Results'][flowline['Well API County Code'] == 
                                         '014'].value_counts())

Test Results for Broomfield Wells:
No Data    323
Pass       319
Fail         1
Name: Integrity Test Results, dtype: int64


In [11]:
B_test_results = flowline['Integrity Test Results'][flowline[
                    'Well API County Code'] == '014']
B_test_results.iplot(kind = 'bar', filename = 'cufflinks/B_test_results')


Aw, snap! We don't have an account for ''. Want to try again? You can authenticate with your email address or username. Sign in is not case sensitive.

Don't have an account? plot.ly

Questions? support@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.

## Extraction Oil & Gas Wells

In [352]:
print('Number of Extraction Operated Wells:', 
      flowline['Integrity Test Results'][flowline['Well Operator'] == 
                                                 'EXTRACTION OIL & GAS INC'].count())

Number of Extraction Operated Wells: 3752


In [353]:
print('Test Results for Extraction Wells:')
print(flowline['Integrity Test Results'][flowline['Well Operator'] == 
                                         'EXTRACTION OIL & GAS INC'].value_counts())

Test Results for Extraction Wells:
Pass       2403
No Data    1334
Fail         15
Name: Integrity Test Results, dtype: int64


# All lines (Assuming there are duplicates)

In [354]:
flowline_nodup = flowline.drop_duplicates()
print('Total Wells After Dropping Duplicates:', flowline_nodup.shape[0])

Total Wells After Dropping Duplicates: 210427


In [362]:
print('Test Results for All Wells After Dropping Duplicates:')
print(flowline_nodup['Integrity Test Results'].value_counts())

Test Results for All Wells After Dropping Duplicates:
No Data    120990
Pass        88901
Fail          536
Name: Integrity Test Results, dtype: int64


## Broomfield (County Code 14)

In [357]:
print('Number of Wells in Broomfield:', 
      flowline_nodup['Integrity Test Results'][flowline_nodup['Well API County Code'] == 
                                         '014'].count())

Number of Wells in Broomfield: 605


In [361]:
print('Test Results for Broomfield Wells:')
print(flowline_nodup['Integrity Test Results'][flowline_nodup['Well API County Code'] == 
                                         '014'].value_counts())

Test Results for Broomfield Wells:
Pass       303
No Data    301
Fail         1
Name: Integrity Test Results, dtype: int64


## Extraction Oil & Gas Wells

In [359]:
print('Number of Extraction Operated Wells:', 
      flowline_nodup['Integrity Test Results'][flowline_nodup['Well Operator'] == 
                                                 'EXTRACTION OIL & GAS INC'].count())

Number of Extraction Operated Wells: 3646


In [360]:
print('Test Results for Extraction Wells:')
print(flowline['Integrity Test Results'][flowline['Well Operator'] == 
                                         'EXTRACTION OIL & GAS INC'].value_counts())

Test Results for Extraction Wells:
Pass       2403
No Data    1334
Fail         15
Name: Integrity Test Results, dtype: int64
